In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import torch
print("pytorch ver . ",torch.__version__)
print("Is CUDA available?",torch.cuda.is_available())
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import torch.utils.data as Data
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import xarray as xr
import os
os.environ['R_HOME'] = '/home/luhung3080/miniconda3/envs/chou/lib/R'
from rpy2.robjects import r, numpy2ri
numpy2ri.activate()
from rpy2.robjects.packages import importr
sinkr = importr('sinkr')
import Nio
print ("pynio ver . ",Nio.__version__)

/home/luhung3080/miniconda3/envs/chou/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pytorch ver .  1.11.0+cu113
Is CUDA available? True
pynio ver .  1.5.5


# Data

In [2]:
data=pd.read_csv('v100_hr_20200101-0930_cut_utf8.csv')

/home/luhung3080/miniconda3/envs/chou/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
data.head(10)

,year,month,day,hour,expdate,stationID,lon,lat,af,tau,cal_PMf,obs_PMf
0,2020,1,2,0,2020-01-01,EPA001,121.7601,25.1292,AF,1,8.01446000,6.000000
1,2020,1,2,1,2020-01-01,EPA001,121.7601,25.1292,AF,2,6.74785000,7.000000
2,2020,1,2,2,2020-01-01,EPA001,121.7601,25.1292,AF,3,6.76573000,5.000000
3,2020,1,2,3,2020-01-01,EPA001,121.7601,25.1292,AF,4,6.52228000,5.000000
4,2020,1,2,4,2020-01-01,EPA001,121.7601,25.1292,AF,5,6.49145000,7.000000
5,2020,1,2,5,2020-01-01,EPA001,121.7601,25.1292,AF,6,6.45912000,6.000000
6,2020,1,2,6,2020-01-01,EPA001,121.7601,25.1292,AF,7,6.49355000,6.000000
7,2020,1,2,7,2020-01-01,EPA001,121.7601,25.1292,AF,8,6.54454000,8.000000
8,2020,1,2,8,2020-01-01,EPA001,121.7601,25.1292,AF,9,6.55783000,11.000000
9,2020,1,2,9,2020-01-01,EPA001,121.7601,25.1292,AF,10,6.55711000,7.000000


In [4]:
data.tail(10)

,year,month,day,hour,expdate,stationID,lon,lat,af,tau,cal_PMf,obs_PMf
1867574,2020,10,4,14,2020-09-30,EPA110,120.2518,23.7535,AF,87,29.50330000,25.000000
1867575,2020,10,4,15,2020-09-30,EPA110,120.2518,23.7535,AF,88,29.68760000,30.000000
1867576,2020,10,4,16,2020-09-30,EPA110,120.2518,23.7535,AF,89,12.16430000,14.000000
1867577,2020,10,4,17,2020-09-30,EPA110,120.2518,23.7535,AF,90,11.52500000,29.000000
1867578,2020,10,4,18,2020-09-30,EPA110,120.2518,23.7535,AF,91,13.51700000,154.000000
1867579,2020,10,4,19,2020-09-30,EPA110,120.2518,23.7535,AF,92,13.03740000,24.000000
1867580,2020,10,4,20,2020-09-30,EPA110,120.2518,23.7535,AF,93,24.22440000,33.000000
1867581,2020,10,4,21,2020-09-30,EPA110,120.2518,23.7535,AF,94,21.99220000,17.000000
1867582,2020,10,4,22,2020-09-30,EPA110,120.2518,23.7535,AF,95,9.85731000,18.000000
1867583,2020,10,4,23,2020-09-30,EPA110,120.2518,23.7535,AF,96,9.53920000,24.000000


# Pytorch lightning

## Data

In [5]:
x1=np.zeros([273,1704])
x2=np.zeros([273,6816])

#x1 (0th~1703th column as x)
for i in range (0,273):
    for j in range (0,71):
        a=np.array(data['obs_PMf'][6816*i+96*j:6816*i+96*j+24])
        for k in range (0,24):
            if a[k]=='\\N' :
                a[k]=np.nan
        for k in range (0,24):
            x1[i][j*24+k]=a[k]

#x2 (1704th~8519th column as x)
for i in range (1,274):
    b=np.array(data['cal_PMf'][6816*i:6816*i+6816])
    for j in range(0,6816):
        if b[j]=='\\N' :
            b[j]=np.nan
    for j in range(0,6816):
        x2[i-1][j]=b[j]
        
print(np.shape(x1))
print(np.shape(x2))

(273, 1704)
(273, 6816)


In [6]:
x1Restruct=sinkr.dineof(x1)
x2Restruct=sinkr.dineof(x2)

[1] "1 EOF ; RMS = 7.21550439"
[1] "1 EOF ; RMS = 7.17572994"
[1] "1 EOF ; RMS = 7.1763972"
[1] "2 EOF ; RMS = 6.36588644"
[1] "2 EOF ; RMS = 6.36731098"
[1] "3 EOF ; RMS = 5.97502741"
[1] "3 EOF ; RMS = 5.97588794"
[1] "4 EOF ; RMS = 5.70473764"
[1] "4 EOF ; RMS = 5.71554036"
[1] "5 EOF ; RMS = 5.45134528"
[1] "5 EOF ; RMS = 5.45004704"
[1] "5 EOF ; RMS = 5.45016246"
[1] "6 EOF ; RMS = 5.29835464"
[1] "6 EOF ; RMS = 5.30034308"
[1] "7 EOF ; RMS = 5.18631531"
[1] "7 EOF ; RMS = 5.18634655"
[1] "8 EOF ; RMS = 5.10529091"
[1] "8 EOF ; RMS = 5.10684981"
[1] "9 EOF ; RMS = 5.00981454"
[1] "9 EOF ; RMS = 5.00984942"
[1] "10 EOF ; RMS = 4.93083121"
[1] "10 EOF ; RMS = 4.93318427"
[1] "11 EOF ; RMS = 4.93959395"
[1] "1 EOF ; RMS = 9.9604114"
[1] "1 EOF ; RMS = 9.94506851"
[1] "1 EOF ; RMS = 9.94412602"
[1] "1 EOF ; RMS = 9.94385791"
[1] "1 EOF ; RMS = 9.94376114"
[1] "1 EOF ; RMS = 9.94372309"
[1] "1 EOF ; RMS = 9.94370733"
[1] "1 EOF ; RMS = 9.9437006"
[1] "2 EOF ; RMS = 9.15939081"
[1] "2 E

In [7]:
x1Restruct_Fun=np.array(x1Restruct[0])
x2Restruct_Fun=np.array(x2Restruct[0])
print(np.shape(x1Restruct_Fun))
print(np.shape(x2Restruct_Fun))

(273, 1704)
(273, 6816)


In [8]:
YRestruct_Fun=np.zeros([269,6816])
for i in range (0,269):
    for j in range (0,1704):
        YRestruct_Fun[i][j]=x1Restruct_Fun[i+1][j]
    for j in range (1704,3408):
        YRestruct_Fun[i][j]=x1Restruct_Fun[i+2][j-1704]
    for j in range (3408,5112):
        YRestruct_Fun[i][j]=x1Restruct_Fun[i+3][j-3408]
    for j in range (5112,6816):
        YRestruct_Fun[i][j]=x1Restruct_Fun[i+4][j-5112]
print(np.shape(YRestruct_Fun))

# 1/2 | 1/3 1/4 1/5 1/6
# .
# .
# .
# 9/22 | 9/23 9/24 9/25 9/26
# 9/23 | 9/24 9/25 9/26 9/27
# 9/24 | 9/25 9/26 9/27 9/28
# 9/25 | 9/26 9/27 9/28 9/29
# 9/26 | 9/27 9/28 9/29 9/30
# 273

(269, 6816)


In [9]:
XRestruct_Fun=np.zeros([269,8520])
for i in range (0,269):
    for j in range (0,1704):
        XRestruct_Fun[i][j]=x1Restruct_Fun[i][j]
    for j in range (1704,8520):
        XRestruct_Fun[i][j]=x2Restruct_Fun[i][j-1704]
print(np.shape(XRestruct_Fun))

(269, 8520)


In [10]:
Xhat=XRestruct_Fun
Yhat=YRestruct_Fun
Xhat_train = np.zeros([243,8520])
Yhat_train = np.zeros([243,6816])
Xhat_val = np.zeros([26,8520])
Yhat_val = np.zeros([26,6816])
Xhat_test = np.zeros([26,8520])
Yhat_test = np.zeros([26,6816])

for i in range (0,243):
    for j in range (0,1704):
        Xhat_train[i][j] = Xhat[i][j]
    for j in range (0,6816):    
        Xhat_train[i][j+1704] = Xhat[i][j+1704]
        Yhat_train[i][j] = Yhat[i][j]
        
for i in range (243,269):
    for j in range (0,1704):
        Xhat_val[i-243][j] = Xhat[i][j]
    for j in range (0,6816):
        Xhat_val[i-243][j+1704] = Xhat[i][j+1704]
        Yhat_val[i-243][j] = Yhat[i][j]     
        
for i in range (243,269):
    for j in range (0,1704):
        Xhat_test[i-243][j] = Xhat[i][j]
    for j in range (0,6816):
        Xhat_test[i-243][j+1704] = Xhat[i][j+1704]
        Yhat_test[i-243][j] = Yhat[i][j]
print('Xhat_train')
print(np.shape(Xhat_train))
print('Yhat_train')
print(np.shape(Yhat_train))
print('Xhat_val')
print(np.shape(Xhat_val))
print('Yhat_val')
print(np.shape(Yhat_val))
print('Xhat_test')
print(np.shape(Xhat_test))
print('Yhat_test')
print(np.shape(Yhat_test))
# Xhat=XRestruct_Fun
# Yhat=YRestruct_Fun
# X1hat_train = np.zeros([244,1704])
# X2hat_train = np.zeros([244,6816])
# Yhat_train = np.zeros([244,6816])
# X1hat_test = np.zeros([25,1704])
# X2hat_test = np.zeros([25,6816])
# Yhat_test = np.zeros([25,6816])
# for i in range (0,244):
#     for j in range (0,1704):
#         X1hat_train[i][j] = Xhat[i][j]
#     for j in range (0,6816):    
#         X2hat_train[i][j] = Xhat[i][j]
#         Yhat_train[i][j] = Yhat[i][j]
# for i in range (244,269):
#     for j in range (0,1704):
#         X1hat_test[i-244][j] = Xhat[i][j]
#     for j in range (0,6816):
#         X2hat_test[i-244][j] = Xhat[i][j]
#         Yhat_test[i-244][j] = Yhat[i][j]
# print('X1hat_train')
# print(np.shape(X1hat_train))
# print('X2hat_train')
# print(np.shape(X2hat_train))
# print('Yhat_train')
# print(np.shape(Yhat_train))
# print('X1hat_test')
# print(np.shape(X1hat_test))
# print('X2hat_test')
# print(np.shape(X2hat_test))
# print('Yhat_test')
# print(np.shape(Yhat_test))
# -----------------------------------------------------------
# 4/3修改前
# Xhat=XRestruct_Fun
# Yhat=YRestruct_Fun
# Xhat_train = np.zeros([244,8520])
# Yhat_train = np.zeros([244,6816])
# Xhat_test = np.zeros([25,8520])
# Yhat_test = np.zeros([25,6816])
# for i in range (0,244):
#     for j in range (0,1704):
#         Xhat_train[i][j] = Xhat[i][j]
#     for j in range (0,6816):
#         Xhat_train[i][j+1704] = Xhat[i][j+1704]
#         Yhat_train[i][j] = Yhat[i][j]
# for i in range (244,269):
#     for j in range (0,1704):
#         Xhat_test[i-244][j] = Xhat[i][j]
#     for j in range (0,6816):
#         Xhat_test[i-244][j+1704] = Xhat[i][j+1704]
#         Yhat_test[i-244][j] = Yhat[i][j]

Xhat_train
(243, 8520)
Yhat_train
(243, 6816)
Xhat_val
(26, 8520)
Yhat_val
(26, 6816)
Xhat_test
(26, 8520)
Yhat_test
(26, 6816)


In [11]:
# x = Xhat_train
# y = Yhat_train
# xt = Xhat_test
# yt = Yhat_test

## Model

In [12]:
x_train = torch.from_numpy(Xhat_train).float()
y_train = torch.from_numpy(Yhat_train).float()
x_val = torch.from_numpy(Xhat_val).float()
y_val = torch.from_numpy(Yhat_val).float()
x_test = torch.from_numpy(Xhat_test).float()
y_test = torch.from_numpy(Yhat_test).float()

In [13]:
class PytorchLightningModel(pl.LightningModule): # 一定要繼承pl.LightningModule
    def __init__(self): # 初始化時可以將基本設定傳入。
        super().__init__()
        self.layer1 = nn.Linear(8520 , 6816)

    def forward(self, x): # 定義模型在forward propagation時如何進行.
        ouput = self.layer1(x)
        return ouput

    def configure_optimizers(self): # 自動訓練時會呼叫此方法來獲取Optimizer.
        return optim.Adam(self.parameters(), lr=1e-2) # 這邊注意要調整的參數是`self.parameters()`
        # optimizer = optim.LBFGS([beta0 , beta1 , beta2] , lr=1e-2)
        # def closure():
        #     yhats_train = model(features1.float() , features2.float(), beta0 , beta1 , beta2)
        #     train_loss = loss(targets.float() , yhats_train)
        #     train_error[epoch] = train_loss
        #     optimizer.zero_grad()
        #     train_loss.backward(retain_graph=True)
        #     return train_loss
        # optimizer.step(closure)    
        # return optim.LBFGS(self.parameters() , lr=1e-2)

#----------------------------------------------------------------------------------------------------------------------------------------------

    def prepare_data(self): # 此方法會在初始化後優先執行。 所以可以在此方法中先將會用到的資料都讀取進來.
        self.train_set = Data.TensorDataset(x_train , y_train) 
        self.test_set = Data.TensorDataset(x_test, y_test)
        self.val_set = Data.TensorDataset(x_val, y_val)
        # logging.info("===== Data is ready... =====")

    # 以下三個方法則是設定進行訓練及驗證時所要使用的Data Loader格式。
    def train_dataloader(self):
        # return Data.DataLoader(dataset=self.train_set, batch_size= self.batch_size, shuffle=True)
        return Data.DataLoader(dataset=self.train_set , batch_size=243 , shuffle=True)

    def test_dataloader(self):
        # return Data.DataLoader(dataset=self.test_set, batch_size= self.batch_size, shuffle=True)
        return Data.DataLoader(dataset=self.test_set , batch_size=26 , shuffle=False)

    def val_dataloader(self):
        #return Data.DataLoader(dataset=self.val_set, batch_size= self.batch_size, shuffle=True)
        return Data.DataLoader(dataset=self.val_set , batch_size=26 , shuffle=False)

#----------------------------------------------------------------------------------------------------------------------------------------------

    def training_step(self, batch, batch_idx): # 定義訓練過程的Step要如何進行
        x, y = batch # 從self.train_dataloader()的Data Loader取一個batch出來。
        output  = self.forward(x)
        loss = nn.MSELoss(reduction='mean')(output, y)
        mae = nn.L1Loss(reduction='mean')(output, y)
        logs = {'loss' : loss , 'MAE_loss' : mae}
#         logs = {'loss' : loss }
        self.log("train_loss_MSE", loss)
        self.log("train_loss_MAE", mae)
#         self.log("train_loss_MAE", loss)
#         return {'loss' : loss , 'log' : logs }
        return {'loss' : loss , 'MAE_loss' : mae , 'log' : logs }

    def test_step(self, batch, batch_idx): #定義 Test 階段
        x, y = batch 
        output  = self.forward(x)
        loss = nn.MSELoss(reduction='mean')(output, y)
        mae = nn.L1Loss(reduction='mean')(output, y)
#         logs = {'loss' : loss }
        logs = {'loss' : loss , 'MAE_loss' : mae}
        self.log("test_loss_MSE", loss)
        self.log("test_loss_MAE", mae)
#         self.log("test_loss_MAE", loss)
#         return {'loss' : loss ,'log' : logs}
        return {'loss' : loss , 'MAE_loss' : mae , 'log' : logs}

    # def test_eposh_end(self, outputs):
    #     avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
    #     logs = {'test_loss': avg_loss}      
    #     #return {'avg_test_loss': avg_loss, 'log': logs, 'progress_bar': logs }
    #     return {'avg_test_loss': avg_loss, 'log': logs}

    def validation_step(self, batch, batch_idx): # 定義Validation如何進行，以這邊為例就再加上了計算Acc.
        x, y = batch 
        output  = self.forward(x)
        loss = nn.MSELoss(reduction='mean')(output, y)
        mae = nn.L1Loss(reduction='mean')(output, y)
        logs = {'loss' : loss , 'MAE_loss' : mae}
#         logs = {'loss' : loss }
        self.log("val_loss_MSE", loss)
        self.log("val_loss_MAE", mae)
#         self.log("val_loss_MAE", loss)
#         return {'loss' : loss ,  'log' : logs}
        return {'loss' : loss , 'MAE_loss' : mae , 'log' : logs}
    
    
    # def validation_epoch_end(self, outputs): # 在Validation的一個Epoch結束後，計算平均的Loss.
    #     avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
    #     logs = {'val_loss': avg_loss}
    #     return {'avg_val_loss': avg_loss, 'log' : logs, 'progress_bar': logs}

In [14]:
logger = TensorBoardLogger("jupyterNB_logs", name="mymodel")
model = PytorchLightningModel() 

# early_stop_callback = EarlyStopping(monitor="train_loss", min_delta=0.00 , patience=1000000 , verbose=True , mode='min', strict=True , check_finite=True , 
#                                                                                 stopping_threshold=150 , divergence_threshold=None , check_on_train_epoch_end=True)
# trainer = pl.Trainer(auto_scale_batch_size="power" ,callbacks=[EarlyStopping(monitor="training_loss")] , max_epochs=200 , gpus=1 , logger=logger)
trainer = pl.Trainer(max_epochs=500 , gpus=1 , logger=logger)
trainer.fit(model)
trainer.test(model)
#trainer.tune(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type   | Params
----------------------------------
0 | layer1 | Linear | 58.1 M
----------------------------------
58.1 M    Trainable params
0         Non-trainable params
58.1 M    Total params
232.317   Total estimated model params size (MB)


/home/luhung3080/miniconda3/envs/chou/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/home/luhung3080/miniconda3/envs/chou/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/home/luhung3080/miniconda3/envs/chou/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1942: PossibleUserWarning: The number

Epoch 0:  50%|████████▌        | 1/2 [00:00<00:00, 44.38it/s, loss=485, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 1:  50%|██████      | 1/2 [00:01<00:01,  1.12s/it, loss=1.01e+06, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 2:  50%|██████▌      | 1/2 [00:02<00:02,  2.28s/it, loss=7.2e+05, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 3:  50%|██████      | 1/2 [00:03<00:03,  3.43s/it, loss=6.51e+05, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 4:  50%|██████      | 1/2 [00:04<00:04,  4.57s/it, loss=7.54e+05, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 5:  50%|██████      | 1/2 [00:05<00:05,  5.73s/it, loss=7.74e+05, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 6:  50%|██████      | 1/2 [00:06<00:06,  6.87s/it, loss=6.99e+05, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 7:  50%|██████      | 1/2 [00:08<00:08,  8.03s/it, loss=6.12e+05, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 8:  50%|██████      | 1/2 [00:09<00:09,  9.17s/it, loss=5.73e+05, v_num=5]
Validation: 0it [00:00,

Epoch 57:  50%|█████▌     | 1/2 [01:05<01:05, 65.60s/it, loss=7.91e+03, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 58:  50%|█████▌     | 1/2 [01:06<01:06, 66.75s/it, loss=6.52e+03, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 59:  50%|█████▌     | 1/2 [01:07<01:07, 67.90s/it, loss=5.56e+03, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 60:  50%|██████      | 1/2 [01:09<01:09, 69.03s/it, loss=5.3e+03, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 61:  50%|█████▌     | 1/2 [01:10<01:10, 70.20s/it, loss=5.27e+03, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 62:  50%|█████▌     | 1/2 [01:11<01:11, 71.35s/it, loss=4.89e+03, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 63:  50%|█████▌     | 1/2 [01:12<01:12, 72.50s/it, loss=4.13e+03, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 64:  50%|█████▌     | 1/2 [01:13<01:13, 73.64s/it, loss=3.43e+03, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 65:  50%|█████▌     | 1/2 [01:14<01:14, 74.78s/it, loss=3.15e+03, v_num=5]
Validation: 0it [00:00,

Epoch 114:  50%|███████▌       | 1/2 [02:10<02:10, 130.36s/it, loss=62, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 115:  50%|██████▌      | 1/2 [02:11<02:11, 131.51s/it, loss=57.3, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 116:  50%|██████▌      | 1/2 [02:12<02:12, 132.64s/it, loss=55.1, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 117:  50%|██████▌      | 1/2 [02:13<02:13, 133.78s/it, loss=54.7, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 118:  50%|██████▌      | 1/2 [02:14<02:14, 134.91s/it, loss=53.1, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 119:  50%|███████▌       | 1/2 [02:16<02:16, 136.06s/it, loss=50, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 120:  50%|███████▌       | 1/2 [02:17<02:17, 137.19s/it, loss=48, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 121:  50%|██████▌      | 1/2 [02:18<02:18, 138.32s/it, loss=47.5, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 122:  50%|██████▌      | 1/2 [02:19<02:19, 139.45s/it, loss=46.6, v_num=5]
Validation: 0it [00:00,

...

Epoch 399:  50%|██████▌      | 1/2 [07:34<07:34, 454.73s/it, loss=7.13, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 400:  50%|███████       | 1/2 [07:35<07:35, 455.86s/it, loss=7.1, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 401:  50%|██████▌      | 1/2 [07:36<07:36, 457.00s/it, loss=7.06, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 402:  50%|██████▌      | 1/2 [07:38<07:38, 458.15s/it, loss=7.02, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 403:  50%|██████▌      | 1/2 [07:39<07:39, 459.32s/it, loss=6.99, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 404:  50%|██████▌      | 1/2 [07:40<07:40, 460.52s/it, loss=6.95, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 405:  50%|██████▌      | 1/2 [07:41<07:41, 461.70s/it, loss=6.91, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 406:  50%|██████▌      | 1/2 [07:42<07:42, 462.84s/it, loss=6.88, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 407:  50%|██████▌      | 1/2 [07:43<07:43, 463.99s/it, loss=6.84, v_num=5]
Validation: 0it [00:00,

Epoch 456:  50%|██████▌      | 1/2 [08:39<08:39, 519.65s/it, loss=5.31, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 457:  50%|██████▌      | 1/2 [08:40<08:40, 520.79s/it, loss=5.28, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 458:  50%|██████▌      | 1/2 [08:41<08:41, 521.93s/it, loss=5.26, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 459:  50%|██████▌      | 1/2 [08:43<08:43, 523.08s/it, loss=5.23, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 460:  50%|███████       | 1/2 [08:44<08:44, 524.22s/it, loss=5.2, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 461:  50%|██████▌      | 1/2 [08:45<08:45, 525.41s/it, loss=5.18, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 462:  50%|██████▌      | 1/2 [08:46<08:46, 526.56s/it, loss=5.15, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 463:  50%|██████▌      | 1/2 [08:47<08:47, 527.71s/it, loss=5.12, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 464:  50%|███████       | 1/2 [08:48<08:48, 528.84s/it, loss=5.1, v_num=5]
Validation: 0it [00:00,

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/luhung3080/miniconda3/envs/chou/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Testing DataLoader 0: 100%|██████████████████████| 1/1 [00:00<00:00, 188.35it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_loss_MAE          6.283151626586914
      test_loss_MSE          67.03026580810547
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss_MSE': 67.03026580810547, 'test_loss_MAE': 6.283151626586914}]

In [15]:
# model_test = PytorchLightningModel()
model_test = PytorchLightningModel.load_from_checkpoint('/home/luhung3080/Desktop/PycharmProjects/NCHUproject/jupyterNB_logs/mymodel/version_5/checkpoints/epoch=499-step=500.ckpt')
trainer = pl.Trainer()  
trainer.test(model_test)
#ver-batch-epoch(train,test)
#48-20-31(150,410) , 51-20-31(25,402) , 55-244-200(21,399) , 57-244-100(147,410) , 88-244-200(21,400)

/home/luhung3080/miniconda3/envs/chou/lib/python3.7/site-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  category=PossibleUserWarning,
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/luhung3080/miniconda3/envs/chou/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1826: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  category=PossibleUserWarning,


Testing DataLoader 0: 100%|███████████████████████| 1/1 [00:00<00:00, 43.61it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_loss_MAE          6.283145427703857
      test_loss_MSE          67.0301513671875
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss_MSE': 67.0301513671875, 'test_loss_MAE': 6.283145427703857}]

@ terminal
(base) luhung3080@luhung3080:

conda activate chou

cd /home/luhung3080/Desktop/PycharmProjects/NCHUproject/jupyterNB_logs/mymodel

tensorboard --logdir=/home/luhung3080/Desktop/PycharmProjects/NCHUproject/jupyterNB_logs/mymodel/version_3 --host=127.0.0.1


In [22]:
class PytorchLightningModel(pl.LightningModule): # 一定要繼承pl.LightningModule
    def __init__(self): # 初始化時可以將基本設定傳入。
        super().__init__()
        self.layer1 = nn.Linear(8520 , 6816)

    def forward(self, x): # 定義模型在forward propagation時如何進行.
        ouput = self.layer1(x)
        return ouput

    def configure_optimizers(self): # 自動訓練時會呼叫此方法來獲取Optimizer.
        return optim.Adam(self.parameters(), lr=1e-5) # 這邊注意要調整的參數是`self.parameters()`

#----------------------------------------------------------------------------------------------------------------------------------------------

    def prepare_data(self): # 此方法會在初始化後優先執行。 所以可以在此方法中先將會用到的資料都讀取進來.
        self.train_set = Data.TensorDataset(x_train , y_train) 
        self.test_set = Data.TensorDataset(x_test, y_test)
        self.val_set = Data.TensorDataset(x_val, y_val)
        # logging.info("===== Data is ready... =====")

    # 以下三個方法則是設定進行訓練及驗證時所要使用的Data Loader格式。
    def train_dataloader(self):
        # return Data.DataLoader(dataset=self.train_set, batch_size= self.batch_size, shuffle=True)
        return Data.DataLoader(dataset=self.train_set , batch_size=243 , shuffle=True)

    def test_dataloader(self):
        # return Data.DataLoader(dataset=self.test_set, batch_size= self.batch_size, shuffle=True)
        return Data.DataLoader(dataset=self.test_set , batch_size=26 , shuffle=False)

    def val_dataloader(self):
        #return Data.DataLoader(dataset=self.val_set, batch_size= self.batch_size, shuffle=True)
        return Data.DataLoader(dataset=self.val_set , batch_size=26 , shuffle=False)

#----------------------------------------------------------------------------------------------------------------------------------------------

    def training_step(self, batch, batch_idx): # 定義訓練過程的Step要如何進行
        x, y = batch # 從self.train_dataloader()的Data Loader取一個batch出來。
        output  = self.forward(x)
        loss = nn.L1Loss(reduction='mean')(output, y)
        logs = {'loss' : loss }
        self.log("train_loss_MAE", loss)
        return {'loss' : loss , 'log' : logs }

    def test_step(self, batch, batch_idx): #定義 Test 階段
        x, y = batch 
        output  = self.forward(x)
        loss = nn.L1Loss(reduction='mean')(output, y)
        logs = {'loss' : loss }
        self.log("test_loss_MAE", loss)
        return {'loss' : loss ,'log' : logs}
    
    # def test_eposh_end(self, outputs):
    #     avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
    #     logs = {'test_loss': avg_loss}      
    #     #return {'avg_test_loss': avg_loss, 'log': logs, 'progress_bar': logs }
    #     return {'avg_test_loss': avg_loss, 'log': logs}

    def validation_step(self, batch, batch_idx): # 定義Validation如何進行，以這邊為例就再加上了計算Acc.
        x, y = batch 
        output  = self.forward(x)
        loss = nn.L1Loss(reduction='mean')(output, y)
        logs = {'loss' : loss }
        self.log("val_loss_MAE", loss)
        return {'loss' : loss ,  'log' : logs}
    
    
    # def validation_epoch_end(self, outputs): # 在Validation的一個Epoch結束後，計算平均的Loss.
    #     avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
    #     logs = {'val_loss': avg_loss}
    #     return {'avg_val_loss': avg_loss, 'log' : logs, 'progress_bar': logs}

In [ ]:
logger = TensorBoardLogger("jupyterNB_logs", name="mymodel")
model = PytorchLightningModel() 
trainer = pl.Trainer(max_epochs=500 , gpus=1 , logger=logger)
trainer.fit(model)
trainer.test(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type   | Params
----------------------------------
0 | layer1 | Linear | 58.1 M
----------------------------------
58.1 M    Trainable params
0         Non-trainable params
58.1 M    Total params
232.317   Total estimated model params size (MB)


Epoch 0:  50%|████████        | 1/2 [00:00<00:00, 52.56it/s, loss=16.5, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 1:  50%|█████████         | 1/2 [00:01<00:01,  1.12s/it, loss=16, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 2:  50%|████████        | 1/2 [00:02<00:02,  2.27s/it, loss=15.6, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 3:  50%|████████        | 1/2 [00:03<00:03,  3.43s/it, loss=15.1, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 4:  50%|████████        | 1/2 [00:04<00:04,  4.57s/it, loss=14.7, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 5:  50%|████████        | 1/2 [00:05<00:05,  5.70s/it, loss=14.3, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 6:  50%|████████        | 1/2 [00:06<00:06,  6.84s/it, loss=13.9, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 7:  50%|████████        | 1/2 [00:07<00:07,  7.99s/it, loss=13.6, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 8:  50%|████████        | 1/2 [00:09<00:09,  9.13s/it, loss=13.2, v_num=9]
Validation: 0it [00:00,

Epoch 29:  50%|███████▌       | 1/2 [00:33<00:33, 33.07s/it, loss=8.15, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 30:  50%|███████▌       | 1/2 [00:34<00:34, 34.20s/it, loss=8.05, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 31:  50%|███████▌       | 1/2 [00:35<00:35, 35.31s/it, loss=7.96, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 32:  50%|███████▌       | 1/2 [00:36<00:36, 36.46s/it, loss=7.88, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 33:  50%|███████▌       | 1/2 [00:37<00:37, 37.61s/it, loss=7.82, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 34:  50%|███████▌       | 1/2 [00:38<00:38, 38.76s/it, loss=7.76, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 35:  50%|███████▌       | 1/2 [00:39<00:39, 39.89s/it, loss=7.71, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 36:  50%|███████▌       | 1/2 [00:41<00:41, 41.00s/it, loss=7.67, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 37:  50%|███████▌       | 1/2 [00:42<00:42, 42.13s/it, loss=7.63, v_num=9]
Validation: 0it [00:00,

Epoch 57:  50%|███████▌       | 1/2 [01:05<01:05, 65.17s/it, loss=7.06, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 58:  50%|███████▌       | 1/2 [01:06<01:06, 66.30s/it, loss=7.04, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 59:  50%|███████▌       | 1/2 [01:07<01:07, 67.42s/it, loss=7.02, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 60:  50%|█████████         | 1/2 [01:08<01:08, 68.55s/it, loss=7, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 61:  50%|███████▌       | 1/2 [01:09<01:09, 69.67s/it, loss=6.98, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 62:  50%|███████▌       | 1/2 [01:10<01:10, 70.79s/it, loss=6.96, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 63:  50%|███████▌       | 1/2 [01:11<01:11, 71.91s/it, loss=6.94, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 64:  50%|███████▌       | 1/2 [01:13<01:13, 73.05s/it, loss=6.92, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 65:  50%|████████        | 1/2 [01:14<01:14, 74.18s/it, loss=6.9, v_num=9]
Validation: 0it [00:00,

Epoch 86:  50%|███████▌       | 1/2 [01:38<01:38, 98.42s/it, loss=6.58, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 87:  50%|███████▌       | 1/2 [01:39<01:39, 99.55s/it, loss=6.57, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 88:  50%|███████       | 1/2 [01:40<01:40, 100.69s/it, loss=6.55, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 89:  50%|███████       | 1/2 [01:41<01:41, 101.83s/it, loss=6.54, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 90:  50%|███████       | 1/2 [01:42<01:42, 102.96s/it, loss=6.53, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 91:  50%|███████       | 1/2 [01:44<01:44, 104.10s/it, loss=6.51, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 92:  50%|███████▌       | 1/2 [01:45<01:45, 105.26s/it, loss=6.5, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 93:  50%|███████       | 1/2 [01:46<01:46, 106.42s/it, loss=6.49, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 94:  50%|███████       | 1/2 [01:47<01:47, 107.55s/it, loss=6.48, v_num=9]
Validation: 0it [00:00,

...

Epoch 371:  50%|██████▌      | 1/2 [07:08<07:08, 428.07s/it, loss=4.41, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 372:  50%|██████▌      | 1/2 [07:09<07:09, 429.21s/it, loss=4.41, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 373:  50%|███████       | 1/2 [07:10<07:10, 430.35s/it, loss=4.4, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 374:  50%|███████       | 1/2 [07:11<07:11, 431.50s/it, loss=4.4, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 375:  50%|██████▌      | 1/2 [07:12<07:12, 432.64s/it, loss=4.39, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 376:  50%|██████▌      | 1/2 [07:13<07:13, 433.79s/it, loss=4.39, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 377:  50%|██████▌      | 1/2 [07:14<07:14, 434.93s/it, loss=4.38, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 378:  50%|██████▌      | 1/2 [07:16<07:16, 436.07s/it, loss=4.38, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 379:  50%|██████▌      | 1/2 [07:17<07:17, 437.21s/it, loss=4.37, v_num=9]
Validation: 0it [00:00,

Epoch 399:  50%|██████▌      | 1/2 [07:40<07:40, 460.60s/it, loss=4.28, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 400:  50%|██████▌      | 1/2 [07:41<07:41, 461.77s/it, loss=4.27, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 401:  50%|██████▌      | 1/2 [07:42<07:42, 462.94s/it, loss=4.27, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 402:  50%|██████▌      | 1/2 [07:44<07:44, 464.17s/it, loss=4.26, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 403:  50%|██████▌      | 1/2 [07:45<07:45, 465.35s/it, loss=4.26, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 404:  50%|██████▌      | 1/2 [07:46<07:46, 466.52s/it, loss=4.25, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 405:  50%|██████▌      | 1/2 [07:47<07:47, 467.67s/it, loss=4.25, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 406:  50%|██████▌      | 1/2 [07:48<07:48, 468.84s/it, loss=4.25, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 407:  50%|██████▌      | 1/2 [07:50<07:50, 470.02s/it, loss=4.24, v_num=9]
Validation: 0it [00:00,

Validation: 0it [00:00, ?it/s]
Epoch 428:  50%|██████▌      | 1/2 [08:15<08:15, 495.22s/it, loss=4.15, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 429:  50%|██████▌      | 1/2 [08:16<08:16, 496.39s/it, loss=4.14, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 430:  50%|██████▌      | 1/2 [08:17<08:17, 497.59s/it, loss=4.14, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 431:  50%|██████▌      | 1/2 [08:18<08:18, 498.75s/it, loss=4.14, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 432:  50%|██████▌      | 1/2 [08:19<08:19, 499.95s/it, loss=4.13, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 433:  50%|██████▌      | 1/2 [08:21<08:21, 501.12s/it, loss=4.13, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 434:  50%|██████▌      | 1/2 [08:22<08:22, 502.32s/it, loss=4.12, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 435:  50%|██████▌      | 1/2 [08:23<08:23, 503.51s/it, loss=4.12, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 436:  50%|██████▌      | 1/2 [08:24<08:24, 504.73s/it, loss=4.11, v

Epoch 456:  50%|██████▌      | 1/2 [08:48<08:48, 528.57s/it, loss=4.03, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 457:  50%|██████▌      | 1/2 [08:49<08:49, 529.73s/it, loss=4.03, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 458:  50%|██████▌      | 1/2 [08:50<08:50, 530.92s/it, loss=4.03, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 459:  50%|██████▌      | 1/2 [08:52<08:52, 532.06s/it, loss=4.02, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 460:  50%|██████▌      | 1/2 [08:53<08:53, 533.22s/it, loss=4.02, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 461:  50%|██████▌      | 1/2 [08:54<08:54, 534.39s/it, loss=4.01, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 462:  50%|██████▌      | 1/2 [08:55<08:55, 535.57s/it, loss=4.01, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 463:  50%|██████▌      | 1/2 [08:56<08:56, 536.73s/it, loss=4.01, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 464:  50%|████████        | 1/2 [08:58<08:58, 538.11s/it, loss=4, v_num=9]
Validation: 0it [00:00,

Validation: 0it [00:00, ?it/s]
Epoch 485:  50%|██████▌      | 1/2 [09:23<09:23, 563.26s/it, loss=3.92, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 486:  50%|██████▌      | 1/2 [09:24<09:24, 564.46s/it, loss=3.92, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 487:  50%|██████▌      | 1/2 [09:25<09:25, 565.63s/it, loss=3.91, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 488:  50%|██████▌      | 1/2 [09:26<09:26, 566.88s/it, loss=3.91, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 489:  50%|██████▌      | 1/2 [09:28<09:28, 568.09s/it, loss=3.91, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 490:  50%|███████       | 1/2 [09:29<09:29, 569.32s/it, loss=3.9, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 491:  50%|███████       | 1/2 [09:30<09:30, 570.52s/it, loss=3.9, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 492:  50%|███████       | 1/2 [09:31<09:31, 571.73s/it, loss=3.9, v_num=9]
Validation: 0it [00:00, ?it/s]
Epoch 493:  50%|██████▌      | 1/2 [09:32<09:32, 572.92s/it, loss=3.89, v

In [ ]:
model_test = PytorchLightningModel.load_from_checkpoint('/home/luhung3080/Desktop/PycharmProjects/NCHUproject/jupyterNB_logs/mymodel/version_9/checkpoints/epoch=499-step=500.ckpt')
trainer = pl.Trainer()  
trainer.test(model_test)